# Bastion hosts
#### ([exercise directory](/tree/notebooks/exercise-06))

In [ ]:
cd /notebooks/exercise-06/


There are many reasons for using bastion hosts:

- security access eg in cloud environment
- vpn eg via Windows hosts
  
The latter case is not interesting as [ansible doesn't support windows as a client platform](https://docs.ansible.com/ansible/latest/os_guide/windows_faq.html#does-ansible-work-with-windows-xp-or-server-2003).

A standard approach is:

- to have an ssh server or a proxy installed on the bastion
- connecto the bastion to the remote network (eg. via vpn)
- configure ssh options in ansible to connect through the bastion
  
We'll do this via two configuration files:

- a standard `ssh_config` where we put the "passthru" configuration
- a simple `ansible.cfg` referencing `ssh_config`
  
This approach allows us:

1. to test the standard ssh connection thru the bastion without messing with ansible
2. keep `ansible.cfg` simple in case we want to reuse them from the intranet (Eg. without traversing the bastion)
  
[![](https://mermaid.ink/img/pako:eNp1UsuOmzAU_RXrLqoZCSwwCSEsKlVqF5XaTbsrjioHLolVuI7AaDLD8O81wekgjbq6r8N5CI9Qmgohh1OnLmf27YckSWWjkezDA14tdqQaKelsestqldcqHHrswhOq7vFxBivq9bFBXtanYtV7cK0bPEjq-_Pv0lCtT8Vbu4awkIcf2dttZu6H4-LK0xaflnq4O5w_Mmwt-o4FqVoREdriKy2h5uHJdH-8Cz-FT7rD6rAEjtkHNlfha-L5VoxV-1IwBz-q3mpD4yjBt5IWYrRnp4eWxZuYi4RvEn59f9uJ-Rbx2LHDNEly239qPiELw_DVhXtlXmJesP8YhQBa7FqlK_dzR0mMSXBqLUrIXVthrYbGSpA0OehwqZTFL5W2poPcdgMGoAZrfj5TeZ8XzGetXPIWnP2md9uLol_GtHeQGyEf4Qq5yFKeiWifxdtkm0ZZtgngGfI423GRZtF2t0_cLYrFFMDLjSHi6T5O0iTdC5G5GiUB4M3S9-WN3p7q9BcAcurg?type=png)](https://mermaid.live/edit#pako:eNp1UsuOmzAU_RXrLqoZCSwwCSEsKlVqF5XaTbsrjioHLolVuI7AaDLD8O81wekgjbq6r8N5CI9Qmgohh1OnLmf27YckSWWjkezDA14tdqQaKelsestqldcqHHrswhOq7vFxBivq9bFBXtanYtV7cK0bPEjq-_Pv0lCtT8Vbu4awkIcf2dttZu6H4-LK0xaflnq4O5w_Mmwt-o4FqVoREdriKy2h5uHJdH-8Cz-FT7rD6rAEjtkHNlfha-L5VoxV-1IwBz-q3mpD4yjBt5IWYrRnp4eWxZuYi4RvEn59f9uJ-Rbx2LHDNEly239qPiELw_DVhXtlXmJesP8YhQBa7FqlK_dzR0mMSXBqLUrIXVthrYbGSpA0OehwqZTFL5W2poPcdgMGoAZrfj5TeZ8XzGetXPIWnP2md9uLol_GtHeQGyEf4Qq5yFKeiWifxdtkm0ZZtgngGfI423GRZtF2t0_cLYrFFMDLjSHi6T5O0iTdC5G5GiUB4M3S9-WN3p7q9BcAcurg)  

## SSH basics

SSH connections follows roughly this basic schema (more information in [RFC4254](https://www.rfc-editor.org/rfc/rfc4253) and further revisions):

1. client connects to server
2. server sends its public key
3. client checks the public key against its `known_hosts` file
4. if the key is not found, the client asks the user to confirm the key
5. if the key is confirmed, the client saves it in `known_hosts`
6. client sends its public key

### SSH Caveats

Host key checking is important. Incorrect validation can lead to:

- persisting an insecure key in your `known_hosts` file
- connecting to the wrong host, that could be a malicious one, and leak credentials and other sensitive information
- connecting to the wrong host (e.g., because someone assigned a wrong IP), and execute commands on it

## ssh_config

Instead of continuously passing options to ssh, we can use [`-F ssh_config`](/edit/notebooks/exercise-06/ssh_config) and put configurations there.


In [ ]:
!cat ssh_config

If we don't use it, we can turn off `GSSApiAuthentication` which attempts may slow down the connection.


### Unsecure by design

Inhibit PKI authentication is insecure by design:

- passwords will surely end in cleartext files
- people eventually do things like the following
  
```
#
# the password is sent to the bastion via a
#   cleartext file.
Match Host 172.25.0.*
    ProxyCommand sshpass -f /home/user/.cleartext-bastion-password ssh -F config jump@bastion -W %h:%p 

```


## Connect to the bastion

Test connectivity to the bastion. Check your host IPs and modify  [`ssh_config`](/edit/notebooks/exercise-06/ssh_config)  accordingly.

**Replace ALL bastion occurrencies, including the one below the BEWARE note**

In [ ]:
fmt=r'{{.NetworkSettings.IPAddress}}'
!docker  -H tcp://172.17.0.1:2375 inspect  ansible-101_bastion_1 --format {fmt} # pass variables *before* commands ;)

#### Exercise

Write the [ssh-copy-id.yml](/edit/notebooks/exercise-06/ssh-copy-id.yml) playbook to install an ssh key to the bastion.

NOTE: a cooler version of this exercise includes generating random passwords and vault pin files using the `lookup` plugin,
like we did in 02_vaults.

Set the bastion credentials to:

- user: `root`
- password `root`
 
Try to do it without watching the previous exercises:

- modify the empty [`ansible.cfg`](/edit/notebooks/exercise-06/ansible.cfg)
    * referencing a pin file 
    * passing [`[ssh_connection]`](http://docs.ansible.com/ansible/intro_configuration.html#openssh-specific-settings) arguments to avoid ssh key mismatches
    * pointing to the local inventory
- store credentials in the encrypted [`vault.yml`](/edit/notebooks/exercise-06/vault.yml). 
- provide an [`inventory`](/edit/notebooks/exercise-06/inventory) file 

You can reuse the old id_ansible key or:

- create a new one and adjust the reference in [` ssh_config`](/edit/notebooks/exercise-06/ssh_config) 
  
Hint:

- if you provide an IdentityFile, password authentication won't work on the `bastion` node;
- you *must* copy ssh id file using password authentication and eventually clean up your known_host file

In [ ]:
# Use this cell to create the pin file and then encrypt the vault

In [ ]:
# Use this cell to test/run  the playbook. You can --limit the execution to the bastion host only.

In [ ]:
!ssh -Fssh_config bastion hostname

# ansible.cfg and ssh_config

In the previous exercise, we used the `[ssh_connection]` stanza to configure ssh connections.

We can instead just set

```
[ssh_connection]
ssh_args = -F ssh_config
```

Write everything in `ssh_config`.


Connecting via bastion in ansible enforcing multiple references to ssh_config

#### Exercise

Uncomment the last lines of  [` ssh_config`](/edit/notebooks/exercise-06/ssh_config) and try to use bastion for connecting to the other hosts
  

In [ ]:
fmt=r'{{.NetworkSettings.IPAddress}}'
!docker  -H tcp://172.17.0.1:2375 inspect  ansible-101_web_1 --format {fmt} # pass variables *before* commands ;)

In [ ]:
!ssh -F ssh_config  root@172.17.0.4 env  # get host ip

#### Exercise

Configure your `ansible.cfg` so that every web host is accessed via the bastion.

  - recycle your dynamic inventory script to access web hosts
  - your id_ansible key should already be on your web hosts 
  - use `ansible -m ping` to check host connectivity
  - run `ps -ef | grep ssh` on your docker host to check all the `ProxyCommand` processes.